In [1]:
import numpy as np
from matplotlib import pyplot as plt
import glob
from climtools import climtools_lib as ctl
import xarray as xr
import matplotlib.cbook as cbook
from scipy import stats
import pickle
import dask.array as da
import os


No DISPLAY variable set. Switching to agg backend


In [2]:
cart_k2 = '/data-hobbes/fabiano/radiative_kernels/Huang/toa/'
finam = 'RRTMG_{}_toa_{}_highR.nc'
kern = xr.load_dataset(cart_k2 + finam.format('wv_lw', 'cld'))
ker = kern.lwkernel
vle=xr.load_dataset(cart_k2+'dp.nc')

In [2]:
#PROVA NUOVI KERNEL
cart_k="/data-hobbes/fabiano/radiative_kernels/Huang_ERA5/ERA5_kernel_{}_TOA.nc"

#['t', 'ts', 'wv_lw', 'wv_sw', 'alb']

vnams = ['ta_dp', 'ts', 'wv_lw_dp', 'wv_sw_dp', 'alb']
allkers = dict()
tips = ['clr', 'cld']
for tip in tips:
    for vna in vnams:
        ker = xr.load_dataset(cart_k.format(vna))
        ker=ker.rename({'latitude': 'lat', 'longitude': 'lon'})
        
        if vna=='ta_dp':
            ker=ker.rename({'level': 'player'})
            vna='t'
        if vna=='wv_lw_dp':
            ker=ker.rename({'level': 'player'})
            vna='wv_lw'
        if vna=='wv_sw_dp':
            ker=ker.rename({'level': 'player'})
            vna='wv_sw'
        if tip=='clr':
            stef=ker.TOA_clr
        else:
            stef=ker.TOA_all
        allkers[(tip, vna)] = stef.assign_coords(month = np.arange(1, 13))

k = allkers[('cld', 't')]
vlevs = xr.load_dataset( '/data-hobbes/fabiano/radiative_kernels/Huang_ERA5/dp_era5.nc')
vlevs=vlevs.rename({'level': 'player','latitude': 'lat', 'longitude': 'lon'})
cose = 100*vlevs.player


In [2]:
cart_out = '/home/rotoli/codici/test3/'
ctl.mkdir(cart_out)

cart_in = '/nas/archive_CMIP6/CMIP6/model-output/EC-Earth-Consortium/EC-Earth3/'
filin_pi = cart_in + 'piControl/atmos/Amon/r1i1p1f1/{}/*nc'
filin_4c = cart_in + 'abrupt-4xCO2/atmos/Amon/r8i1p1f1_r25/{}/*nc'
feedbacks = dict()


In [3]:
filist = glob.glob(filin_4c.format('ta'))
filist.sort()
var= xr.open_mfdataset(filist, chunks = {'time': 12}, use_cftime=True)
var=var['ta']

A=(var.plev/var)*(9.81/1005)
laps1=(var.diff(dim='plev'))*A  #derivata sulla verticale = laspe-rate

laps1=laps1.where(laps1<=-2)
mask = laps1/laps1


In [4]:
mask

<xarray.DataArray (time: 1980, plev: 18, lat: 73, lon: 144)>
dask.array<truediv, shape=(1980, 18, 73, 144), dtype=float64, chunksize=(12, 1, 73, 144), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float64 -90.0 -87.5 -85.0 -82.5 -80.0 ... 82.5 85.0 87.5 90.0
  * plev     (plev) float64 9.25e+04 8.5e+04 7e+04 6e+04 ... 1e+03 500.0 100.0

In [4]:

#calcolo ts_anom e gtas e plank surf
vnam = 'ts'
filist = glob.glob(filin_4c.format(vnam))
filist.sort()
var = xr.open_mfdataset(filist, chunks = {'time': 12}, use_cftime=True)

filist = glob.glob(filin_pi.format(vnam))
filist.sort()
pivar = xr.open_mfdataset(filist, chunks = {'time': 12}, use_cftime=True)[vnam]
pivar = pivar.sel(time = slice('2540-01-01', '2689-12-31'))
var = var.sel(time = slice('1850-01-01', '1999-12-31'))
piok=pivar.groupby('time.month').mean()
piok = ctl.regrid_dataset(piok, k.lat, k.lon)
var = ctl.regrid_dataset(var[vnam], k.lat, k.lon) 
#piok=ctl.running_mean(piok, 252).drop('time')
#piok=(piok.rolling(time=252, center=True).mean()).drop('time')
#piok['time'] = var['time']
anoms = var.groupby('time.month') - piok
#anoms.compute()
ts_anom = anoms
gtas = ctl.global_mean(anoms).groupby('time.year').mean('time')

# kernel = kernel = allkers[('cld', 'ts')]  #AGGIUSTA
# dRt = xr.apply_ufunc(lambda x, ker: x*ker, anoms.groupby('time.month'), kernel, dask = 'allowed').groupby('time.year').mean('time')

# dRt_glob = ctl.global_mean(dRt)
# feedbacks = dRt_glob.compute()
gtas= gtas.groupby((gtas.year-1) // 10 * 10).mean()
# feedback=feedbacks.groupby((feedbacks.year-1) // 10 * 10).mean()
# res = stats.linregress(gtas, feedback)



Regridding...
Regridding completed in 0:00:02.766195
Regridding...
Regridding completed in 0:00:00.194025


/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]


In [6]:
anoms

<xarray.DataArray (time: 1800, lat: 73, lon: 144)>
dask.array<sub, shape=(1800, 73, 144), dtype=float32, chunksize=(1, 73, 144), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1850-01-16 12:00:00 ... 1999-12-16 12:00:00
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12

In [31]:

ps = xr.open_mfdataset('/data-hobbes/fabiano/radiative_kernels/ps_ece3/ps_Amon_EC-Earth3_stabilization-hist-1990_r1i1p1f1_gr_199?01-199?12.nc')
psclim = ps.groupby('time.month').mean()
psye = psclim['ps'].mean('month')
psye_rg = ctl.regrid_dataset(psye, k.lat, k.lon).compute()
wid_mask = np.empty([len(vlevs.player)] + list(psye_rg.shape))

for ila in range(len(psye_rg.lat)):
    #print(ila)
  for ilo in range(len(psye_rg.lon)):
            ind = np.where((psye_rg[ila, ilo].values/100. - vlevs.player.values) > 0)[0][0]
            wid_mask[:ind, ila, ilo] = np.nan
            wid_mask[ind, ila, ilo] = psye_rg[ila, ilo].values/100. - vlevs.player.values[ind]
            wid_mask[ind+1:, ila, ilo] = (vlevs.mean('month')).dp.values[ind+1:, ila, ilo]

wid_mask = xr.DataArray(wid_mask, dims = k.dims[1:], coords = k.drop('month').coords)


Regridding...
Regridding completed in 0:00:01.133881


In [ ]:

filist = glob.glob(filin_4c.format('ta'))
filist.sort()
temp = xr.open_mfdataset(filist, chunks = {'time':12}, use_cftime=True)
#temp=(temp).interp(plev = cose)

A=(temp.plev/temp['ta'])*(9.81/1005)
laps1=(temp['ta'].diff(dim='plev'))*A  #derivata sulla verticale = laspe-rate

laps1=laps1.where(laps1<=-2)
mask1 = laps1/laps1

In [5]:

filist = glob.glob(filin_4c.format('ta'))
filist.sort()
temp = xr.open_mfdataset(filist, chunks = {'time':12}, use_cftime=True)
temp=(temp).interp(plev = cose)

A=(temp.plev/temp['ta'])*(9.81/1005)
laps1=(temp['ta'].diff(dim='plev'))*A  #derivata sulla verticale = laspe-rate

laps1=laps1.where(laps1<=-2)
mask = laps1/laps1

filist = glob.glob(filin_pi.format('ta'))
filist.sort()
pivar = xr.open_mfdataset(filist, chunks = {'time': 12}, use_cftime=True)['ta']
pivar = pivar.sel(time = slice('2540-01-01', '2689-12-31'))
pivar=pivar.groupby('time.month').mean()
piok = ctl.regrid_dataset(pivar, k.lat, k.lon)
ta_abs_pi = (piok).interp(plev = cose)

Regridding...
Regridding completed in 0:00:01.247557


In [9]:
temp

<xarray.Dataset>
Dimensions:    (time: 1980, bnds: 2, lon: 144, lat: 73, plev: 19)
Coordinates:
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * lon        (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat        (lat) float64 -90.0 -87.5 -85.0 -82.5 ... 82.5 85.0 87.5 90.0
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(12, 2), meta=np.ndarray>
    ta         (time, plev, lat, lon) float32 dask.array<chunksize=(12, 1, 73, 144), meta=np.ndarray>
Attributes: (12/47)
    CDI:                    Climate Data Interface version 1.9.6 (http://mpim...
    history:                Wed Nov 22 16:34:04 2023: cdo remap,grid_25.des,r...
    source:                 EC-Earth3 (2019): \naerosol: none\natmos: IFS cy3...
    institution:            AEMET, Spain; BSC, Spain; CNR-ISAC, Italy; DMI, D...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    ...                     ...
    variant_label:          r8i1p1f1
    license:                CMIP6 model data produced by EC-Earth-Consortium ...
    cmor_version:           3.4.0
    branch_time_in_parent:  252017.
    tracking_id:            hdl:21.14100/34237f34-29f9-41ad-aafe-96e94fb662e6
    CDO:                    Climate Data Operators version 1.9.6 (http://mpim...

In [36]:
mm = mask.groupby('time.month').mean().compute()
ctl.plot_multimap_contour([mm.isel(month = 0, player = i) for i in range(len(mm.player))], cbar_range = (0, 1), subtitles = [f'{int(lev.values/100)} hPa' for lev in mm.player])

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point

Adding cyclic point



[<Figure size 1500x1200 with 19 Axes>, <Figure size 1500x1200 with 19 Axes>]

In [7]:
ta_abs_pi

<xarray.DataArray (month: 12, player: 37, lat: 73, lon: 144)>
dask.array<transpose, shape=(12, 37, 73, 144), dtype=float32, chunksize=(1, 37, 73, 144), chunktype=numpy.ndarray>
Coordinates:
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    plev     (player) float32 1e+05 9.75e+04 9.5e+04 ... 300.0 200.0 100.0
  * player   (player) float32 1e+03 975.0 950.0 925.0 900.0 ... 5.0 3.0 2.0 1.0
Attributes:
    regrid_method:  bilinear

In [14]:
mask

<xarray.DataArray (time: 1980, player: 37, lat: 73, lon: 144)>
dask.array<truediv, shape=(1980, 37, 73, 144), dtype=float32, chunksize=(12, 37, 73, 144), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float64 -90.0 -87.5 -85.0 -82.5 -80.0 ... 82.5 85.0 87.5 90.0
    plev     (player) float32 1e+05 9.75e+04 9.5e+04 ... 300.0 200.0 100.0
  * player   (player) float32 1e+03 975.0 950.0 925.0 900.0 ... 5.0 3.0 2.0 1.0

In [ ]:
filist = glob.glob(filin_4c.format('ta'))
filist.sort()
var = xr.open_mfdataset(filist, chunks = {'time': 12}, use_cftime=True)
var = var['ta']
var = var.sel(time = slice('1850-01-01', '1999-12-31')) #MODIFICA
var = ctl.regrid_dataset(var, k.lat, k.lon)
anoms_ok = var - piok
anoms_ok = (anoms_ok*mask).interp(plev = cose)
kernel = allkers[('cld', 't')]
anoms_lr = (anoms_ok - ts_anom)  
anoms_unif = (anoms_ok - anoms_lr)
    
dRt_unif = (xr.apply_ufunc(lambda x, ker, wid: x*ker*wid, anoms_unif.groupby('time.month'), kernel, vlevs.dp/100., dask = 'allowed')).sum('player').groupby('time.year').mean('time')
dRt_lr = (xr.apply_ufunc(lambda x, ker, wid: x*ker*wid, anoms_lr.groupby('time.month'), kernel, vlevs.dp/100., dask = 'allowed')).sum('player').groupby('time.year').mean('time')

dRt_unif_glob = ctl.global_mean(dRt_unif)
dRt_lr_glob = ctl.global_mean(dRt_lr)
feedbacks_atmo = dRt_unif_glob.compute()
feedbacks_lr = dRt_lr_glob.compute()
feedback1=feedbacks_lr.groupby((feedbacks_lr.year-1) // 10 * 10).mean()
feedback2=feedbacks_atmo.groupby((feedbacks_atmo.year-1) // 10 * 10).mean()
res1 = stats.linregress(gtas, feedback1)
res2 = stats.linregress(gtas, feedback2)

In [17]:
filist = glob.glob(filin_4c.format('hus'))
filist.sort()
var = xr.open_mfdataset(filist, chunks = {'time': 12}, use_cftime=True)['hus']

var = var.sel(time = slice('1850-01-01', '1999-12-31'))
var = ctl.regrid_dataset(var, k.lat, k.lon)

var_int1 = (var*mask1).interp(plev = cose)

Regridding...
Regridding completed in 0:00:00.179037


In [6]:
filist = glob.glob(filin_4c.format('hus'))
filist.sort()
var = xr.open_mfdataset(filist, chunks = {'time': 12}, use_cftime=True)['hus']
filist = glob.glob(filin_pi.format('hus'))
filist.sort()
pivar = xr.open_mfdataset(filist, chunks = {'time': 12}, use_cftime=True)['hus']
pivar = pivar.sel(time = slice('2540-01-01', '2689-12-31'))
var = var.sel(time = slice('1850-01-01', '1999-12-31'))
var = ctl.regrid_dataset(var, k.lat, k.lon)
piok= pivar.groupby('time.month').mean()
piok = ctl.regrid_dataset(piok, k.lat, k.lon)

var_int = (var*mask).interp(plev = cose)
piok_int = piok.interp(plev = cose)

#qo = piok.interp(plev = cose)
anoms= var_int.groupby('time.month') - piok_int
#anoms = (anoms*mask).interp(plev = cose)
Rv = 487.5 # gas constant of water vapor
Lv = 2.5e+06 # latent heat of water vapor
kernel_lw = allkers[('cld', 'wv_lw')]
kernel_sw = allkers[('cld', 'wv_sw')]
kernel = kernel_lw + kernel_sw
#coso = (anoms/piok_int) * (ta_abs_pi**2) * Rv/Lv

Regridding...
Regridding completed in 0:00:00.200656
Regridding...
Regridding completed in 0:00:01.077912


/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]


In [10]:
var_into = (var*mask).interp(plev = cose)

In [34]:
PROVA=ctl.global_mean(var_int1)-ctl.global_mean(var_int)
plt.plot(PROVA)
plt.savefig('nonva.pdf')

In [ ]:
import spectfbcalc_lib as sfc

In [13]:

# anoms= var - piok
# anoms = (anoms*mask).interp(plev = cose)
# Rv = 487.5 # gas constant of water vapor
# Lv = 2.5e+06 # latent heat of water vapor
# kernel_lw = allkers[('cld', 'wv_lw')]
# kernel_sw = allkers[('cld', 'wv_sw')]
# kernel = kernel_lw + kernel_sw
# coso = (anoms/qo) * (ta_abs_pi**2) * Rv/Lv
coso = (anoms.groupby('time.month')/piok_int).groupby('time.month') * (ta_abs_pi**2) * Rv/Lv
dRt=(xr.apply_ufunc(lambda x, ker, wid: x*ker*wid, coso.groupby('time.month'), kernel, vlevs.dp / 100 , dask = 'allowed')).sum('player').groupby('time.year').mean('time')
        
#dRt = (xr.apply_ufunc(lambda x, ker: x*ker, coso.groupby('time.month'), kernel, dask = 'allowed') * Rv/Lv * vlevs.dp / 100.).sum('player').groupby('time.year').mean('time')
dRt_glob = ctl.global_mean(dRt)
print('compute dRt')
feedbacks = dRt_glob.compute()
feedback=feedbacks.groupby((feedbacks.year-1) // 10 * 10).mean()
res_wv = stats.linregress(gtas, feedback)
res_wv

/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]
/home/rotoli/miniconda3/envs/ctl4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]


compute dRt


IndexError: index 0 is out of bounds for axis 0 with size 0

In [15]:
coso

<xarray.DataArray (time: 1800, lat: 73, lon: 144, player: 37)>
dask.array<truediv, shape=(1800, 73, 144, 37), dtype=float32, chunksize=(1, 73, 144, 37), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1850-01-16 12:00:00 ... 1999-12-16 12:00:00
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
    plev     (player) float32 1e+05 9.75e+04 9.5e+04 ... 300.0 200.0 100.0
  * player   (player) float32 1e+03 975.0 950.0 925.0 900.0 ... 5.0 3.0 2.0 1.0

In [27]:
fatto=xr.open_dataarray("/home/rotoli/codici/test/dRt_water-vapor_global_cld_climatology-HUANGkernels.nc",  use_cftime=True)

In [28]:
fatto-feedbacks

<xarray.DataArray (year: 150)>
array([-124563.78820061, -212566.70210768, -267745.36924282,
       -325231.69515335, -385613.20117256, -406638.88579884,
       -429957.00960548, -457320.08727812, -444116.48386345,
       -486765.14008987, -487879.68190774, -510276.62232733,
       -512622.37134644, -529331.53366048, -530165.69328339,
       -543869.95624264, -545058.3773391 , -555013.34473206,
       -566151.34846305, -569449.65803001, -581897.35336679,
       -578705.3594096 , -602486.93428899, -616123.37212882,
       -606521.30877154, -608505.33069804, -613527.47955378,
       -628132.94463062, -641549.20462149, -626283.43505965,
       -636120.42663464, -627376.15563941, -639260.69424574,
       -642907.99315768, -624802.52200255, -640177.65824331,
       -655964.34810452, -654911.06890232, -649822.2011933 ,
       -660407.95823091, -677615.20076649, -683719.94571276,
       -696737.30278361, -700053.14886439, -706235.18806128,
       -693442.66936598, -682051.63390968, -694615.39919486,
       -701631.83178244, -714409.92984907, -726167.24044052,
       -729793.61012564, -737937.30800395, -729893.15718582,
       -736689.35530104, -738938.28512704, -733820.54512554,
       -740881.60330638, -753449.65226734, -749653.90429655,
...
       -818679.06933836, -819137.44068041, -809381.44193883,
       -813654.60398589, -810096.98774076, -822999.35888732,
       -812395.42128933, -830956.00190793, -836648.91830245,
       -824293.82755053, -835622.8130104 , -820987.01546391,
       -840746.48586075, -815061.39331268, -803721.12866223,
       -809738.18797931, -816123.62883473, -818443.76843242,
       -815786.08989006, -816743.44242513, -822335.93107279,
       -825975.27368649, -816154.01167901, -825208.25128114,
       -817548.31681364, -822227.60747042, -844309.33106074,
       -843377.27731234, -838629.07434619, -839785.13142146,
       -838829.51247231, -826245.02258903, -825947.39450091,
       -829554.36091842, -835020.64343535, -826137.13738586,
       -828581.68739613, -842039.76455798, -842924.9224846 ,
       -850220.40370438, -857979.00373776, -852111.82667119,
       -863261.96988402, -838732.12042533, -852374.11767316,
       -867614.98886845, -837809.17408905, -856110.74362264,
       -858089.06390088, -848779.45670645, -835089.77696767,
       -860497.1106757 , -866429.34148635, -854498.65523884,
       -891780.37841964, -853802.4725432 , -839490.81897235,
       -858410.05109291, -860723.57584646, -863988.79035009])
Coordinates:
  * year     (year) int64 1850 1851 1852 1853 1854 ... 1995 1996 1997 1998 1999

In [ ]:
#PRENDERE I KERNEL

def ker(cart_k, cart_out):
  """
    Loads and processes climate kernel datasets, and saves specific datasets to pickle files.

    Parameters:
    -----------
    cart_k : str
        Path template to the kernel dataset files. 
        Placeholders should be formatted as `{}` to allow string formatting.
        
    cart_out : str
        Path template to save the outputs. 

    Returns:
    --------
    allkers : dict
        A dictionary containing the loaded and processed kernels.
    
    Additional Outputs:
    -------------------
    The function also saves three objects as pickle files in a predefined output directory:
      - `vlevs.p`: The vertical levels data from the 'dp.nc' file.
      - `k.p`: The longwave kernel data corresponding to cloudy-sky temperature ('cld', 't').
      - `cose.p`: A scaled version (100x) of the 'player' variable from the vertical levels data.
  """


  vnams = ['t', 'ts', 'wv_lw', 'wv_sw', 'alb']
  tips = ['clr', 'cld']

  allkers = dict()

  for tip in tips:
     for vna in vnams:
         ker = xr.load_dataset(cart_k.format(vna, tip))

         allkers[(tip, vna)] = ker.assign_coords(month = np.arange(1, 13))

  vlevs = xr.load_dataset(cart_k + 'dp.nc')
  k = allkers[('cld', 't')].lwkernel
  pickle.dump(vlevs, open(cart_out + 'vlevs.p', 'wb')) #save vlevs
  pickle.dump(k, open(cart_out + 'k.p', 'wb')) #save k
  cose = 100*vlevs.player
  pickle.dump(cose, open(cart_out + 'cose.p', 'wb'))
  pickle.dump(allkers, open(cart_out + 'allkers.p', 'wb'))
  return allkers


In [ ]:
##CALCOLO PIMEAN PROVA

def clim(filin_pi, cart_k, cart_out, allvars, MEAN=True, chu=12):
    # Check if the kernel file exists, if not, call the ker() function
    pimean = dict()
    k_file_path = os.path.join(cart_out, 'k.p')
    if not os.path.exists(k_file_path):
        print("Kernel file not found. Running ker() to generate it.")
        ker(cart_k)  # Ensure that ker() is properly defined elsewhere in the code

    if allvars=='alb':
       allvars='rsus rsds'.split()
       if MEAN==True:
        for vnam in allvars:
            k=pickle.load(open(cart_out + 'k.p', 'rb'))#prendi da cart_out
            filist = glob.glob(filin_pi.format(vnam))
            filist.sort()

            var = xr.open_mfdataset(filist, chunks = {'time': chu}, use_cftime=True)

            var_mean = var.mean('time')

            var_mean = ctl.regrid_dataset(var_mean, k.lat, k.lon)

            pimean[vnam] = var_mean[vnam].compute()

            pimean[vnam].to_netcdf(cart_out+ "amoc_all_1000_" + vnam + ".nc", format="NETCDF4")

#poi calcolo albedo
        pimean[('alb')] = pimean[('rsus')]/pimean[('rsds')]

        pimean[('alb')].to_netcdf(cart_out+ "amoc_all_1000_alb.nc", format="NETCDF4")
       else:
        for vnam in allvars:
            filist = glob.glob(filin_pi.format(vnam))
            filist.sort()
            pivar = xr.open_mfdataset(filist, chunks = {'time': chu}, use_cftime=True)
            pivar = pivar.sel(time = slice('2540-01-01', '2689-12-31')) #MODIFICA
            piok = ctl.regrid_dataset(pivar[vnam], k.lat, k.lon)
            piok=ctl.running_mean(piok, 252)
            piok.to_netcdf(cart_out+ "piok_" + vnam + "_21y.nc", format="NETCDF4")
     
        pivar['alb']=pivar['rsus']/pivar['rdsd']
        piok = ctl.regrid_dataset(pivar['alb'], k.lat, k.lon)
        piok=ctl.running_mean(piok, 252)
        piok.to_netcdf(cart_out+ "piok_alb_21y.nc", format="NETCDF4") 

    if MEAN==True:
      for vnam in allvars:
        k=pickle.load(open(cart_out + 'k.p', 'rb'))#prendi da cart_out
        filist = glob.glob(filin_pi.format(vnam))
        filist.sort()

        var = xr.open_mfdataset(filist, chunks = {'time': chu}, use_cftime=True)

        var_mean = var.mean('time')

        var_mean = ctl.regrid_dataset(var_mean, k.lat, k.lon)

        pimean[vnam] = var_mean[vnam].compute()

        pimean[vnam].to_netcdf(cart_out+ "amoc_all_1000_" + vnam + ".nc", format="NETCDF4")
    else:
      for vnam in allvars:
        filist = glob.glob(filin_pi.format(vnam))
        filist.sort()
        pivar = xr.open_mfdataset(filist, chunks = {'time': chu}, use_cftime=True)
        pivar = pivar.sel(time = slice('2540-01-01', '2689-12-31')) #MODIFICA
        piok = ctl.regrid_dataset(pivar[vnam], k.lat, k.lon)
        piok=ctl.running_mean(piok, 252)
        piok.to_netcdf(cart_out+ "piok_" + vnam + "_21y.nc", format="NETCDF4")
     

In [ ]:
##CALCOLO PIMEAN

# def clim(filin_pi:str, cart_k:str, cart_out:str, MEAN=True, chu=12):
#     """
#     Processes climate data from multiple NetCDF files, computes mean or running averages, 
#     regrids variables, and optionally computes albedo. Outputs are saved as NetCDF files.

#     Parameters:
#     -----------
#     filin_pi : str
#         Path template for input NetCDF piControl files, with a placeholder for variable names. 
#         Placeholders should be formatted as `{}` to allow string formatting.

#     cart_k : str
#         Path template to the kernel dataset files. 
#         Placeholders should be formatted as `{}` to allow string formatting.
                
#     cart_out : str
#         Path template to save the outputs. 

#     MEAN : bool, optional (default=True)
#         If True, computes the mean over the entire time dimension for each variable. 
#         If False, processes time slices and computes running means for each variable.
    
#     chu : int, optional (default=12)
#         Chunk size for loading data using xarray to optimize memory usage.

#     Additional Outputs:
#     -------------------
#     1. Mean Computation (`MEAN=True`):
#         - Saves the mean of each variable regridded to match the kernel's latitude 
#           and longitude as NetCDF files.
#         - Calculates and saves albedo (`alb`), defined as the ratio of `rsus` to `rsds`.

#     2. Running Mean Computation (`MEAN=False`):
#         - Filters the time range to 2540-01-01 to 2689-12-31.
#         - Saves 21-year running averages of each variable and the calculated albedo 
#           regridded to match the kernel's latitude and longitude.

#     Notes:
#     ------
#        Outputs filenames follow the format `amoc_all_1000_{variable}.nc` for means 
#        and `piok_{variable}_21y.nc` for running averages.
#     """
#     # Check if the kernel file exists, if not, call the ker() function
#     k_file_path = os.path.join(cart_out, 'k.p')
#     if not os.path.exists(k_file_path):
#         print("Kernel file not found. Running ker() to generate it.")
#         ker(cart_k, cart_out)  # Ensure that ker() is properly defined elsewhere in the code

#     allvars = 'rlutcs rsutcs clt hus rlut rsds rsus rsut ta tas ts'.split()
#     pimean = dict()
#     k=pickle.load(open(cart_out + 'k.p', 'rb'))#prendi da cart_out
#     if MEAN==True:
#       for vnam in allvars:
#         filist = glob.glob(filin_pi.format(vnam))
#         filist.sort()
#         var = xr.open_mfdataset(filist, chunks = {'time': chu}, use_cftime=True)
#         var_mean = var.mean('time')

#         var_mean = ctl.regrid_dataset(var_mean, k.lat, k.lon)

#         pimean[vnam] = var_mean[vnam].compute()

#         pimean[vnam].to_netcdf(cart_out+ "amoc_all_1000_" + vnam + ".nc", format="NETCDF4")

# #poi calcolo albedo
#       pimean[('alb')] = pimean[('rsus')]/pimean[('rsds')]

#       pimean[('alb')].to_netcdf(cart_out+ "amoc_all_1000_alb.nc", format="NETCDF4")
   
   
#     else:
#       for vnam in allvars:
#         filist = glob.glob(filin_pi.format(vnam))
#         filist.sort()
#         pivar = xr.open_mfdataset(filist, chunks = {'time': chu}, use_cftime=True)
#         pivar = pivar.sel(time = slice('2540-01-01', '2689-12-31')) #MODIFICA
#         piok = ctl.regrid_dataset(pivar[vnam], k.lat, k.lon)
#         piok=ctl.running_mean(piok, 252)
#         piok.to_netcdf(cart_out+ "piok_" + vnam + "_21y.nc", format="NETCDF4")
     
#       pivar['alb']=pivar['rsus']/pivar['rdsd']
#       piok = ctl.regrid_dataset(pivar['alb'], k.lat, k.lon)
#       piok=ctl.running_mean(piok, 252)
#       piok.to_netcdf(cart_out+ "piok_alb_21y.nc", format="NETCDF4")


##CALCOLO PIMEAN

def clim(filin_pi, cart_k, cart_out, MEAN=True, chu=12):
    """
    Processes climate data from multiple NetCDF files, computes mean or running averages, 
    regrids variables, and optionally computes albedo. Outputs are saved as NetCDF files.

    Parameters:
    -----------
    filin_pi : str
        Path template for input NetCDF piControl files, with a placeholder for variable names. 
        Placeholders should be formatted as `{}` to allow string formatting.

    cart_k : str
        Path template to the kernel dataset files. 
        Placeholders should be formatted as `{}` to allow string formatting.
                
    cart_out : str
        Path template to save the outputs. 

    MEAN : bool, optional (default=True)
        If True, computes the mean over the entire time dimension for each variable. 
        If False, processes time slices and computes running means for each variable.
    
    chu : int, optional (default=12)
        Chunk size for loading data using xarray to optimize memory usage.

    Additional Outputs:
    -------------------
    1. Mean Computation (`MEAN=True`):
        - Saves the mean of each variable regridded to match the kernel's latitude 
          and longitude as NetCDF files.
        - Calculates and saves albedo (`alb`), defined as the ratio of `rsus` to `rsds`.

    2. Running Mean Computation (`MEAN=False`):
        - Filters the time range to 2540-01-01 to 2689-12-31.
        - Saves 21-year running averages of each variable and the calculated albedo 
          regridded to match the kernel's latitude and longitude.

    Notes:
    ------
       Outputs filenames follow the format `amoc_all_1000_{variable}.nc` for means 
       and `piok_{variable}_21y.nc` for running averages.
    """
    # Check if the kernel file exists, if not, call the ker() function
    k_file_path = os.path.join(cart_out, 'k.p')
    if not os.path.exists(k_file_path):
        print("Kernel file not found. Running ker() to generate it.")
        ker(cart_k)  # Ensure that ker() is properly defined elsewhere in the code

    allvars = 'rlutcs rsutcs clt hus rlut rsds rsus rsut ta tas ts'.split()
    pimean = dict()

    if MEAN==True:
      for vnam in allvars:
        k=pickle.load(open(cart_out + 'k.p', 'rb'))#prendi da cart_out
        filist = glob.glob(filin_pi.format(vnam))
        filist.sort()

        var = xr.open_mfdataset(filist, chunks = {'time': chu}, use_cftime=True)

        var_mean = var.mean('time')

        var_mean = ctl.regrid_dataset(var_mean, k.lat, k.lon)

        pimean[vnam] = var_mean[vnam].compute()

        pimean[vnam].to_netcdf(cart_out+ "amoc_all_1000_" + vnam + ".nc", format="NETCDF4")

#poi calcolo albedo
      pimean[('alb')] = pimean[('rsus')]/pimean[('rsds')]

      pimean[('alb')].to_netcdf(cart_out+ "amoc_all_1000_alb.nc", format="NETCDF4")
   
   
    else:
      for vnam in allvars:
        filist = glob.glob(filin_pi.format(vnam))
        filist.sort()
        pivar = xr.open_mfdataset(filist, chunks = {'time': chu}, use_cftime=True)
        pivar = pivar.sel(time = slice('2540-01-01', '2689-12-31')) #MODIFICA
        piok = ctl.regrid_dataset(pivar[vnam], k.lat, k.lon)
        piok=ctl.running_mean(piok, 252)
        piok.to_netcdf(cart_out+ "piok_" + vnam + "_21y.nc", format="NETCDF4")
     
      pivar['alb']=pivar['rsus']/pivar['rdsd']
      piok = ctl.regrid_dataset(pivar['alb'], k.lat, k.lon)
      piok=ctl.running_mean(piok, 252)
      piok.to_netcdf(cart_out+ "piok_alb_21y.nc", format="NETCDF4")


In [ ]:
##calcolare tropopausa (Reichler 2003) 

def mask_atm(filin_4c, chu=12):
    """
    Generates a mask for atmospheric temperature data based on the lapse rate threshold.
    as in (Reichler 2003) 

    Parameters:
    -----------
    filin_4c : str
        Path template for input NetCDF files, with a placeholder for the variable name (e.g., 'ta').
        Placeholders should be formatted as `{}` to allow string formatting.

    chu : int, optional (default=12)
        Chunk size for loading data using xarray to optimize memory usage.

    Returns:
    --------
    mask : xarray.DataArray
        A mask array where:
        - Values are 1 where the lapse rate (`laps1`) is less than or equal to -2 K/km.
        - Values are NaN elsewhere.
    """

    filist = glob.glob(filin_4c.format('ta'))
    filist.sort()
    temp = xr.open_mfdataset(filist, chunks = {'time': chu}, use_cftime=True)

    A=(temp.plev/temp['ta'])*(9.81/1005)
    laps1=(temp['ta'].diff(dim='plev'))*A  #derivata sulla verticale = laspe-rate

    laps1=laps1.where(laps1<=-2)
    mask = laps1/laps1
    return mask

In [ ]:
### Mask for surf pressure

def mask_pres(dir, cart_out, cart_k):
    """
    Generates a pressure mask based on climatological surface pressure and vertical levels.

    Parameters:
    -----------
    dir : str
        Path to the input NetCDF file or directory containing surface pressure (`ps`) data.
        The data is expected to include a time dimension for monthly averaging.

    cart_out : str
        Path template to save the outputs. 
        
    cart_k : str
        Path template to the kernel dataset files, required for generating vertical level data.
        Used by the `ker()` function if the kernel data is missing.

    Returns:
    --------
    wid_mask : xarray.DataArray
        A 3D mask array (pressure level, latitude, longitude) where:
        - Values are NaN for pressure levels above the surface pressure.
        - Values represent vertical level pressure differences for levels below the surface pressure.
    """
    
    # Check if the kernel file exists, if not, call the ker() function
    k_file_path = os.path.join(cart_out, 'k.p')
    if not os.path.exists(k_file_path):
        print("Kernel file not found. Running ker() to generate it.")
        ker(cart_k)  # Ensure that ker() is properly defined elsewhere in the code
    k=pickle.load(open(cart_out + 'k.p', 'rb'))#prendi da cart_out
    vlevs=pickle.load(open(cart_out + 'vlevs.p', 'rb'))#prendi da cart_out
    ps = xr.open_mfdataset(dir)
    psclim = ps.groupby('time.month').mean()
    psye = psclim['ps'].mean('month')
    psye_rg = ctl.regrid_dataset(psye, k.lat, k.lon).compute()
    wid_mask = np.empty([len(vlevs.player)] + list(psye_rg.shape))

    for ila in range(len(psye_rg.lat)):
    #print(ila)
        for ilo in range(len(psye_rg.lon)):
            ind = np.where((psye_rg[ila, ilo].values/100. - vlevs.player.values) > 0)[0][0]
            wid_mask[:ind, ila, ilo] = np.nan
            wid_mask[ind, ila, ilo] = psye_rg[ila, ilo].values/100. - vlevs.player.values[ind]
            wid_mask[ind+1:, ila, ilo] = vlevs.dp.values[ind+1:]

    wid_mask = xr.DataArray(wid_mask, dims = k.dims[1:], coords = k.drop('month').coords)
    return wid_mask
 

In [ ]:
def pliq(T):
    pliq = 0.01 * np.exp(54.842763 - 6763.22 / T - 4.21 * np.log(T) + 0.000367 * T + np.tanh(0.0415 * (T - 218.8)) * (53.878 - 1331.22 / T - 9.44523 * np.log(T) + 0.014025 * T))
    return pliq

def pice(T):
    pice = np.exp(9.550426 - 5723.265 / T + 3.53068 * np.log(T) - 0.00728332 * T) / 100.0
    return pice

def dlnws(T):
    """
    Calculates 1/(dlnq/dT_1K).
    """
    pliq0 = pliq(T)
    pice0 = pice(T)

    T1 = T + 1.0
    pliq1 = pliq(T1)
    pice1 = pice(T1)
    
    # Use np.where to choose between pliq and pice based on the condition T >= 273
    if isinstance(T, xr.DataArray) and isinstance(T.data, da.core.Array):
        ws = da.where(T >= 273, pliq0, pice0)    # Dask equivalent of np.where is da.where
        ws1 = da.where(T1 >= 273, pliq1, pice1)
    else:
        ws = np.where(T >= 273, pliq0, pice0)
        ws1 = np.where(T1 >= 273, pliq1, pice1)
    
    # Calculate the inverse of the derivative dws
    dws = ws / (ws1 - ws)

    if isinstance(dws, np.ndarray):
        dws = ctl.transform_to_dataarray(T, dws, 'dlnws')
    
    return dws


In [ ]:
#calcolo ts_anom e gtas e plank surf
#aggiustare chunck e date

def fb_planck_surf(filin_4c, filin_pi, cart_out, cart_k, MEAN=True, chu=12):
    """
    Computes the surface Planck feedback using temperature anomalies and precomputed kernels.

    Parameters:
    -----------
    filin_4c : str
        Path template for input NetCDF files, with a placeholder for the variable name (e.g., 'ts').
        Placeholders should be formatted as `{}` to allow string formatting.

    filin_pi : str
        Path template for the preindustrial (PI) temperature files, required to compute anomalies.

    cart_out : str
        Path template to save the outputs. 

    cart_k : str
        Path template to the kernel dataset files, used by the `ker()` function if kernel data is missing.

    MEAN : bool, optional (default=True)
        If True, uses mean climatological data from precomputed PI files (`amoc_all_1000_ts.nc`).
        If False, uses running mean data from precomputed PI files (`piok_ts_21y.nc`).

    chu : int, optional (default=12)
        Chunk size for loading data using xarray to optimize memory usage.

    Returns:
    --------
    None
        The function does not return a value but saves the following outputs to the `cart_out` directory:
        - Global annual mean surface Planck feedback for clear-sky and all-sky conditions.
        - Temperature anomalies (`ts_anom`) relative to the PI climatology as NetCDF files.
        - Global temperature anomaly series (`gtas`) as NetCDF files.

   
    """   

    feedbacks=dict()
    # Check if the kernel file exists, if not, call the ker() function
    k_file_path = os.path.join(cart_out, 'k.p')
    if not os.path.exists(k_file_path):
        print("Kernel file not found. Running ker() to generate it.")
        allkers=ker(cart_k)  # Ensure that ker() is properly defined elsewhere in the code
    k=pickle.load(open(cart_out + 'k.p', 'rb'))#prendi da cart_out
    vlevs=pickle.load(open(cart_out + 'vlevs.p', 'rb'))#prendi da cart_out

    filist = glob.glob(filin_4c.format('ts'))
    filist.sort()
    var = xr.open_mfdataset(filist, chunks = {'time': chu}, use_cftime=True)
    var = var.sel(time = slice('1850-01-01', '1999-12-31')) #MODIFICA
    var = ctl.regrid_dataset(var['ts'], k.lat, k.lon)

    if MEAN==True:
     path = os.path.join(cart_out, 'amoc_all_1000_ts.nc')
     if not os.path.exists(path):
       clim(filin_pi, cart_k, cart_out, MEAN, chu)
     piok=xr.open_dataarray(cart_out+"amoc_all_1000_ts.nc", use_cftime=True)
    else:
     path = os.path.join(cart_out, 'piok_ts_21y.nc')
     if not os.path.exists(path):
       clim(filin_pi, cart_k, cart_out, MEAN, chu)
     piok=xr.open_dataarray(cart_out+"piok_ts_21y.nc", chunks = {'time': chu}, use_cftime=True) 
     piok=piok.drop('time')
     piok['time'] = var['time']
    anoms = var - piok
 
    for tip in ['clr', 'cld']:
     kernel = allkers[(tip, 'ts')].lwkernel

     dRt = xr.apply_ufunc(lambda x, ker: x*ker, anoms.groupby('time.month'), kernel, dask = 'allowed').groupby('time.year').mean('time')
     dRt_glob = ctl.global_mean(dRt)
     feedbacks[( tip, 'planck-surf')] = dRt_glob.compute()
     feedbacks[( tip, 'planck-surf')].to_netcdf(cart_out+ "dRt_plank-surf_global_" +tip +".nc", format="NETCDF4")


    anoms.compute()
    ts_anom = anoms
    ts_anom.to_netcdf(cart_out+ "ts_anom.nc", format="NETCDF4")
    gtas = ctl.global_mean(anoms).groupby('time.year').mean('time')
    gtas.to_netcdf(cart_out+ "gtas.nc", format="NETCDF4")



In [ ]:
#CALCOLO PLANK-ATMO E LAPSE RATE CON TROPOPAUSA VARIABILE (DA CONTROLLARE)

def fb_plank_atm_lr(filin_4c, filin_pi, cart_out, cart_k, dir, MEAN = True, chu=12):
   """
    Computes atmospheric Planck and lapse-rate feedbacks using temperature anomalies, kernels, and masking.

    Parameters:
    -----------
    filin_4c : str
        Path template for input NetCDF files, with a placeholder for the variable name (e.g., 'ta').
        Placeholders should be formatted as `{}` to allow string formatting.

    filin_pi : str
        Path template for the preindustrial (PI) temperature files, required to compute anomalies.

    cart_out : str
        Output directory where precomputed files (`k.p`, `cose.p`, `vlevs.p`) and results are stored.

    cart_k : str
        Path template to the kernel dataset files, used by the `ker()` function if kernel data is missing.

    dir : str
        Directory containing surface pressure (`ps`) data required for pressure masking.

    MEAN : bool, optional (default=True)
        If True, uses mean climatological data from precomputed PI files (`amoc_all_1000_ta.nc`).
        If False, uses running mean data from precomputed PI files (`piok_ta_21y.nc`).

    chu : int, optional (default=12)
        Chunk size for loading data using xarray to optimize memory usage.

    Returns:
    --------
    None
        The function does not return a value but saves the following outputs to the `cart_out` directory:
        - Global annual mean Planck and lapse-rate feedbacks for clear-sky and all-sky conditions.
        - Preindustrial absolute temperature profile interpolated to the kernel levels.
        - Anomaly and mask data for further analysis.

    """
   
   feedbacks=dict()
   # Check if the kernel file exists, if not, call the ker() function
   k_file_path = os.path.join(cart_out, 'k.p')
   if not os.path.exists(k_file_path):
        print("Kernel file not found. Running ker() to generate it.")
        allkers=ker(cart_k)  # Ensure that ker() is properly defined elsewhere in the code   

   k=pickle.load(open(cart_out + 'k.p', 'rb'))#prendi da cart_out
   cose=pickle.load(open(cart_out + 'cose.p', 'rb'))
   vlevs=pickle.load(open(cart_out + 'vlevs.p', 'rb'))#prendi da cart_out
   
   filist = glob.glob(filin_4c.format('ta'))
   filist.sort()
   var = xr.open_mfdataset(filist, chunks = {'time': chu}, use_cftime=True)
   var = var['ta']
   var = var.sel(time = slice('1850-01-01', '1999-12-31')) #MODIFICA
   var = ctl.regrid_dataset(var, k.lat, k.lon)

   if MEAN==True:
      path = os.path.join(cart_out, 'amoc_all_1000_ta.nc')
      if not os.path.exists(path):
       clim(filin_pi, cart_k, cart_out, MEAN, chu)      
      piok=xr.open_dataarray(cart_out+"amoc_all_1000_ta.nc",  use_cftime=True)
   else:
      path = os.path.join(cart_out, 'piok_ta_21y.nc')
      if not os.path.exists(path):
       clim(filin_pi, cart_k, cart_out, MEAN, chu)
      piok=xr.open_dataarray(cart_out+"piok_ta_21y.nc", chunks = {'time': chu}, use_cftime=True) 
      piok=piok.drop('time')
      piok['time'] = var['time']
   anoms_ok = var - piok

   ta_abs_pi = piok.interp(plev = cose)
   ta_abs_pi.to_netcdf(cart_out+ "ta_abs_pi.nc", format="NETCDF4")
   mask=mask_atm(filin_4c, chu)
   wid_mask=mask_pres(dir, cart_out, cart_k)
   anoms_ok = (anoms_ok*mask).interp(plev = cose)
   ts_anom=pickle.load(open(cart_out + 'ts_anom.p', 'rb'))#prendi da cart_out
 

   for tip in ['clr','cld']:
     kernel = allkers[(tip, 't')].lwkernel
     anoms_lr = (anoms_ok - ts_anom)  
     anoms_unif = (anoms_ok - anoms_lr)
    
     dRt_unif = (xr.apply_ufunc(lambda x, ker, wid: x*ker*wid, anoms_unif.groupby('time.month'), kernel, wid_mask/100., dask = 'allowed')).sum('player').groupby('time.year').mean('time')
     dRt_lr = (xr.apply_ufunc(lambda x, ker, wid: x*ker*wid, anoms_lr.groupby('time.month'), kernel, wid_mask/100., dask = 'allowed')).sum('player').groupby('time.year').mean('time')

     dRt_unif_glob = ctl.global_mean(dRt_unif)
     dRt_lr_glob = ctl.global_mean(dRt_lr)
     feedbacks[( tip, 'planck-atmo')] = dRt_unif_glob.compute()
     feedbacks[( tip, 'lapse-rate')] = dRt_lr_glob.compute()
    
     feedbacks[( tip, 'planck-atmo')].to_netcdf(cart_out+ "dRt_plank-atmo_global_" +tip +".nc", format="NETCDF4")
     feedbacks[( tip, 'lapse-rate')].to_netcdf(cart_out+ "dRt_lapse-rate_global_" +tip +".nc", format="NETCDF4")



In [ ]:
#CONTO ALBEDO

def fb_albedo(filin_4c, filin_pi, cart_out, cart_k, MEAN=True, chu=12):
  """
    Computes the albedo feedback using surface albedo anomalies and precomputed kernels.

    Parameters:
    -----------
    filin_4c : str
        Path template for input NetCDF files, with a placeholder for the variable name (e.g., 'rsus', 'rsds').
        Placeholders should be formatted as `{}` to allow string formatting.

    filin_pi : str
        Path template for the preindustrial (PI) albedo files, required to compute anomalies.

    cart_out : str
        Output directory where precomputed files (`k.p`) and results are stored.

    cart_k : str
        Path template to the kernel dataset files, used by the `ker()` function if kernel data is missing.

    MEAN : bool, optional (default=True)
        If True, uses mean climatological data from precomputed PI files (`amoc_all_1000_alb.nc`).
        If False, uses running mean data from precomputed PI files (`piok_alb_21y.nc`).

    chu : int, optional (default=12)
        Chunk size for loading data using xarray to optimize memory usage.

    Returns:
    --------
    None
        The function does not return a value but saves the following outputs to the `cart_out` directory:
        - Global annual mean albedo feedback for clear-sky and all-sky conditions.
        - Albedo anomaly data for further analysis.

  """
  # Check if the kernel file exists, if not, call the ker() function
  k_file_path = os.path.join(cart_out, 'k.p')
  if not os.path.exists(k_file_path):
        print("Kernel file not found. Running ker() to generate it.")
        allkers=ker(cart_k)  # Ensure that ker() is properly defined elsewhere in the code   

 
  k=pickle.load(open(cart_out + 'k.p', 'rb'))#prendi da cart_out

  feedbacks=dict()
  filist_1 = glob.glob(filin_4c.format('rsus'))
  filist_1.sort()
  var_rsus = xr.open_mfdataset(filist_1, chunks = {'time': chu}, use_cftime=True)['rsus']
  filist_2 = glob.glob(filin_4c.format('rsds'))
  filist_2.sort()
  var_rsds = xr.open_mfdataset(filist_2, chunks = {'time': chu}, use_cftime=True)['rsds']
  var = var_rsus/var_rsds
  var = var.sel(time = slice('1850-01-01', '1999-12-31')) #MODIFICA
  var = ctl.regrid_dataset(var, k.lat, k.lon)

  if MEAN==True:
    path = os.path.join(cart_out, 'amoc_all_1000_alb.nc')
    if not os.path.exists(path):
     clim(filin_pi, cart_k, cart_out, MEAN, chu)
    piok=xr.open_dataarray(cart_out+"amoc_all_1000_alb.nc",  use_cftime=True)
  else:
   path = os.path.join(cart_out, 'piok_alb_21y.nc')
   if not os.path.exists(path):
    clim(filin_pi, cart_k, cart_out, MEAN, chu)
   piok=xr.open_dataarray(cart_out+"piok_alb_21y.nc", chunks = {'time': chu}, use_cftime=True) 
   piok=piok.drop('time')
   piok['time'] = var['time']

# Removing inf and nan from alb
  piok = piok.where(piok > 0., 0.)
  var = var.where(var > 0., 0.)
  anoms =  var - piok

  for tip in [ 'clr','cld']:
    kernel = allkers[(tip, 'alb')].swkernel

    dRt = xr.apply_ufunc(lambda x, ker: x*ker, anoms.groupby('time.month'), kernel, dask = 'allowed').groupby('time.year').mean('time')
    dRt_glob = ctl.global_mean(dRt).compute()
    feedbacks[( tip, 'albedo')] = 100*dRt_glob
    feedbacks[( tip, 'albedo')].to_netcdf(cart_out+ "dRt_albedo_global_" +tip +".nc", format="NETCDF4")

In [ ]:
##CALCOLO W-V
def fb_wv(filin_4c, filin_pi, cart_out, cart_k, dir, MEAN=True, chu=12):
    
   """
    Computes the water vapor feedback using specific humidity (hus) anomalies, precomputed kernels,
    and vertical integration.

    Parameters:
    -----------
    filin_4c : str
        Path template for input NetCDF files containing climate data, with a placeholder for the variable
        name (e.g., 'hus'). Placeholders should be formatted as `{}` to allow string formatting.

    filin_pi : str
        Path template for preindustrial (PI) climate files, required for computing anomalies.

    cart_out : str
        Output directory where precomputed files (e.g., `k.p`, `cose.p`, `vlevs.p`) and results are stored.

    cart_k : str
        Path template for kernel dataset files, used by the `ker()` function if kernel data is missing.

    dir : str
        Path to the pressure-level data directory, used for creating masks and vertical integration.

    MEAN : bool, optional (default=True)
        If True, uses mean climatological data from precomputed PI files (`amoc_all_1000_hus.nc`).
        If False, uses running mean data from precomputed PI files (`piok_hus_21y.nc`).

    chu : int, optional (default=12)
        Chunk size for loading data using xarray to optimize memory usage.

    Returns:
    --------
    None
        The function does not return a value but saves the following outputs to the `cart_out` directory:
        - Global annual mean water vapor feedback for clear-sky and all-sky conditions.
        - Intermediate water vapor anomalies for further analysis.

   """

   # Check if the kernel file exists, if not, call the ker() function
   k_file_path = os.path.join(cart_out, 'k.p')
   if not os.path.exists(k_file_path):
        print("Kernel file not found. Running ker() to generate it.")
        allkers=ker(cart_k)  # Ensure that ker() is properly defined elsewhere in the code   

   k=pickle.load(open(cart_out + 'k.p', 'rb'))#prendi da cart_out

   feedbacks=dict()
   allkers=ker(cart_k)
   k=pickle.load(open(cart_out + 'k.p', 'rb'))#prendi da cart_out
   cose=pickle.load(open(cart_out + 'cose.p', 'rb'))
   vlevs=pickle.load(open(cart_out + 'vlevs.p', 'rb'))#prendi da cart_out
   ta_abs_pi=xr.open_dataarray(cart_out+"ta_ans_pi.nc",  use_cftime=True)
   mask=mask_atm(filin_4c)
   wid_mask=mask_pres(dir)
   filist = glob.glob(filin_4c.format('hus')) 
   filist.sort()
   var = xr.open_mfdataset(filist, chunks = {'time': chu}, use_cftime=True)
   var = var['hus']
   var = var.sel(time = slice('1850-01-01', '1999-12-31')) #MODIFICA
   var = ctl.regrid_dataset(var, k.lat, k.lon)

   if MEAN==True:
     path = os.path.join(cart_out, 'amoc_all_1000_hus.nc')
     if not os.path.exists(path):
      clim(filin_pi, cart_k, cart_out, MEAN, chu)
     piok=xr.open_dataarray(cart_out+"amoc_all_1000_hus.nc", use_cftime=True)
   else:
     path = os.path.join(cart_out, 'piok_hus_21y.nc')
     if not os.path.exists(path):
      clim(filin_pi, cart_k, cart_out, MEAN, chu)
     piok=xr.open_dataarray(cart_out+"piok_hus_21y.nc", chunks = {'time': chu}, use_cftime=True) 
     piok=piok.drop('time')
     piok['time'] = var['time']
 

   var_int = (var*mask).interp(plev = cose)
   piok_int = piok.interp(plev = cose)
   anoms_ok3 = xr.apply_ufunc(lambda x, mean: np.log(x) - np.log(mean), var_int, piok_int , dask = 'allowed')
   coso3= xr.apply_ufunc(lambda x, ta: x*ta, anoms_ok3, dlnws(ta_abs_pi), dask = 'allowed') #(using dlnws)
   for tip in ['clr','cld']:
      print(tip)
      kernel_lw = allkers[(tip, 'wv_lw')].lwkernel
      kernel_sw = allkers[(tip, 'wv_sw')].swkernel
      kernel = kernel_lw + kernel_sw
      dRt = (xr.apply_ufunc(lambda x, ker, wid: x*ker*wid, coso3.groupby('time.month'), kernel, wid_mask/100., dask = 'allowed')).sum('player').groupby('time.year').mean('time')
      dRt_glob = ctl.global_mean(dRt)
      feedbacks[(tip, 'water-vapor')] = dRt_glob.compute()
      feedbacks[( tip, 'water-vapor')].to_netcdf(cart_out+ "dRt_water-vapor_global_" +tip +".nc", format="NETCDF4")



In [ ]:
##CALCOLO EFFETTIVO DEI FEEDBACK(INSERISCO IL RICHIAMO ALLE ALTRE FUNZIONI?)
def calc_fb(filin_4c, filin_pi, cart_out, cart_k, dir, MEAN=True, chu=12):
    """
    Computes climate feedback coefficients by combining feedback components (Planck, albedo, water vapor, etc.)
    with global temperature anomalies and performs a linear regression analysis.

    Parameters:
    -----------
    filin_4c : str
        Path template for input NetCDF files containing climate data, with a placeholder for the variable
        name (e.g., 'ta'). Placeholders should be formatted as `{}` to allow string formatting.

    filin_pi : str
        Path template for preindustrial (PI) climate files, required for computing anomalies.

    cart_out : str
        Output directory where intermediate feedback data and results are stored.

    cart_k : str
        Path template for kernel dataset files, used by feedback functions like `fb_planck_surf`.

    dir : str
        Path to the pressure-level data directory, used for additional computations such as water vapor feedback.

    MEAN : bool, optional (default=True)
        If True, computes anomalies relative to the mean climatology from precomputed PI files.
        If False, computes anomalies relative to running mean PI data.

    chu : int, optional (default=12)
        Chunk size for loading data using xarray to optimize memory usage.

    Returns:
    --------
    dict
        A dictionary (`fb_coef`) containing feedback coefficients for each combination of:
        - Feedback component: `planck-surf`, `planck-atmo`, `lapse-rate`, `water-vapor`, `albedo`.
        - Sky condition: `clr` (clear-sky), `cld` (cloudy-sky).
        Feedback coefficients are computed as the slope of the linear regression between global temperature
        anomalies (`gtas`) and feedback response (`dRt`).

    Notes:
    ------
     **Feedback Computation**:
       - This function relies on precomputed feedback outputs (e.g., `dRt_planck-surf_global_clr.nc`).
       - If the required feedback files are missing, corresponding feedback functions (`fb_planck_surf`, `fb_plank_atm_lr`, etc.) are automatically called.

    Outputs:
    --------
    The function returns a dictionary with regression results for each feedback component and sky condition.
    Intermediate feedback outputs (e.g., `dRt_planck-surf_global_clr.nc`) are generated as needed.
    """    
    
    path = os.path.join(cart_out, "dRt_planck-surf_global_clr.nc")
    if not os.path.exists(path):
        fb_planck_surf(filin_4c, filin_pi, cart_out, cart_k, MEAN=True, chu=12)
    path = os.path.join(cart_out, "dRt_planck-atmo_global_clr.nc")
    if not os.path.exists(path):
        fb_plank_atm_lr(filin_4c, filin_pi, cart_out, cart_k, dir, MEAN, chu)
    path = os.path.join(cart_out, "dRt_albedo_global_clr.nc")
    if not os.path.exists(path):
        fb_albedo(filin_4c, filin_pi, cart_out, cart_k, MEAN, chu)
    path = os.path.join(cart_out, "dRt_water-vapor_global_clr.nc")
    if not os.path.exists(path):
        fb_wv(filin_4c, filin_pi, cart_out, cart_k, dir, MEAN, chu)    
    fbnams = ['planck-surf', 'planck-atmo', 'lapse-rate', 'water-vapor', 'albedo']
    fb_coef = dict()
    feedbacks=dict()
    gtas=xr.open_dataarray(cart_out+"gtas.nc",  use_cftime=True)
    gtas= gtas.groupby((gtas.year-1) // 10 * 10).mean()

    for tip in ['clr', 'cld']:
        for fbn in fbnams:

            feedbacks=xr.open_dataarray(cart_out+"dRt_" +fbn+"_global_"+tip+ ".nc",  use_cftime=True)
            feedback=feedbacks.groupby((feedbacks.year-1) // 10 * 10).mean()

            res = stats.linregress(gtas, feedback)
            fb_coef[(tip, fbn)] = res
    
    return(fb_coef)


In [ ]:

# ###CLOUD FEEDBACK shell 2008
def fb_cloud(filin_4c:str, filin_4c1:str, cart_out:str, cart_k:str, chu=12):
    """
    Computes the cloud feedback coefficient based on the Shell (2008) approach,
    using radiative fluxes, clear-sky fluxes, and feedback components. 

    Parameters:
    -----------
    filin_4c : str
        Path template for NetCDF files containing radiative flux variables (e.g., `rlut`, `rsut`).
        The template should include `{}` for formatting with variable names.

    filin_4c1 : str
        Path template for NetCDF files containing clear-sky radiative flux variables (`rsutcs`, `rlutcs`).
        The template should include `{}` for formatting with variable names.

    cart_out : str
        Output directory where intermediate and precomputed datasets (e.g., `k.p`, `gtas.nc`) are stored.
        
    cart_k : str
        Path template for kernel dataset files, used by the `ker()` function if kernel data is missing.

    chu : int, optional (default=12)
        Chunk size for loading data using xarray to optimize memory usage.

    Returns:
    --------
    tuple
        A tuple containing:
        - `fb_cloud` (float): The computed cloud feedback coefficient, representing the change in radiative
          forcing due to cloud processes.
        - `fb_cloud_err` (float): The estimated uncertainty (standard error) of the cloud feedback coefficient.

    References:
    -----------
    - Shell et al. (2008): Framework for decomposing climate feedbacks.
    - Caldwell et al. (2016): Analysis of cloud feedback uncertainties.

    """
    # Check if the kernel file exists, if not, call the ker() function
    k_file_path = os.path.join(cart_out, 'k.p')
    if not os.path.exists(k_file_path):
        print("Kernel file not found. Running ker() to generate it.")
        allkers=ker(cart_k)  # Ensure that ker() is properly defined elsewhere in the code   
    k=pickle.load(open(cart_out + 'k.p', 'rb'))#prendi da cart_out

    fbnams = ['planck-surf', 'planck-atmo', 'lapse-rate', 'water-vapor', 'albedo']
    pimean=dict()
    fb_coef=dict()
    fb_coef=calc_fb()#questo così non funziona, ci vanno gli input(che però dovrei aggiungere anche alla funzione)

    filist = glob.glob(filin_4c.format('rlut'))
    filist.sort()
    rlut = xr.open_mfdataset(filist, chunks = {'time': chu}, use_cftime=True)['rlut']
    rlut = rlut.sel(time = slice('1850-01-01', '1999-12-31')) #MODIFICA
 
    filist = glob.glob(filin_4c.format('rsut'))
    filist.sort()
    rsut = xr.open_mfdataset(filist, chunks = {'time': chu}, use_cftime=True)['rsut']
    rsut = rsut.sel(time = slice('1850-01-01', '1999-12-31')) #MODIFICA

    filist = glob.glob(filin_4c1.format('rsutcs')) 
    filist.sort()
    rsutcs = xr.open_mfdataset(filist, chunks = {'time': chu}, use_cftime=True)['rsutcs']
    rsutcs= ctl.regrid_dataset(rsutcs, k.lat, k.lon)
    rsutcs = rsutcs.sel(time = slice('1850-01-01', '1999-12-31'))#MODIFICA

    filist = glob.glob(filin_4c1.format('rlutcs'))
    filist.sort()
    rlutcs = xr.open_mfdataset(filist, chunks = {'time': chu}, use_cftime=True)['rlutcs']
    rlutcs = ctl.regrid_dataset(rlutcs, k.lat, k.lon)
    rlutcs = rlutcs.sel(time = slice('1850-01-01', '1999-12-31'))#MODIFICA


    N = - rlut - rsut
    N0 = - rsutcs - rlutcs

    crf = (N0 - N) 
    crf = crf.groupby('time.year').mean('time')

    N = N.groupby('time.year').mean('time')
    N0 = N0.groupby('time.year').mean('time')

    crf_glob = ctl.global_mean(crf).compute()
    N_glob = ctl.global_mean(N).compute()
    N0_glob = ctl.global_mean(N0).compute()

    crf_glob= crf_glob.groupby((crf_glob.year-1) // 10 * 10).mean(dim='year')
    N_glob=N_glob.groupby((N_glob.year-1) // 10 * 10).mean(dim='year')
    N0_glob=N0_glob.groupby((N0_glob.year-1) // 10 * 10).mean(dim='year')

    gtas=xr.open_dataarray(cart_out+"gtas.nc",  use_cftime=True)
    gtas= gtas.groupby((gtas.year-1) // 10 * 10).mean()
    res_N = stats.linregress(gtas, N_glob)
    res_N0 = stats.linregress(gtas, N0_glob)
    res_crf = stats.linregress(gtas, crf_glob)

    pimean[('rlutcs')] =pickle.load(open(cart_out + 'amoc_all_1000_rlutcs.p', 'rb')) #già fatto il regrid per loro
    pimean[('rsutcs')] =pickle.load(open(cart_out + 'amoc_all_1000_rsutcs.p', 'rb'))
    pimean[('rsut')] =pickle.load(open(cart_out + 'amoc_all_1000_rsut.p', 'rb'))
    pimean[('rlut')] =pickle.load(open(cart_out + 'amoc_all_1000_rlut.p', 'rb'))

    F0 = res_N0.intercept + pimean[('rlutcs')] + pimean[('rsutcs')] 
    F = res_N.intercept + pimean[('rlut')] + pimean[('rsut')]
    F0.compute()
    F.compute()

    F_glob = ctl.global_mean(F)
    F0_glob = ctl.global_mean(F0)
    F_glob = F_glob.compute()
    F0_glob = F0_glob.compute()

    fb_cloud = -res_crf.slope + np.nansum([fb_coef[( 'clr', fbn)].slope - fb_coef[('cld', fbn)].slope for fbn in fbnams]) #letto in Caldwell2016

    fb_cloud_err = np.sqrt(res_crf.stderr**2 + np.nansum([fb_coef[('cld', fbn)].stderr**2 for fbn in fbnams]))
    return(fb_cloud, fb_cloud_err)